# ASL Letter Recognition

## Setup

In [1]:
WORKPLACE_PATH = 'Tensorflow/workplace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKPLACE_PATH + '/annotations'
IMAGE_PATH = WORKPLACE_PATH + '/images'
MODEL_PATH = WORKPLACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKPLACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

## Generate Labels

In [2]:
labels = [{'name':'a', 'id':1},
          {'name':'b', 'id':2},
          {'name':'c', 'id':3},
          {'name':'d', 'id':4},
          {'name':'e', 'id':5},
          {'name':'f', 'id':6},
          {'name':'g', 'id':7},
          {'name':'h', 'id':8},
          {'name':'i', 'id':9},
          {'name':'j', 'id':10},
          {'name':'k', 'id':11},
          {'name':'l', 'id':12},
          {'name':'m', 'id':13},
          {'name':'n', 'id':14},
          {'name':'o', 'id':15},
          {'name':'p', 'id':16},
          {'name':'q', 'id':17},
          {'name':'r', 'id':18},
          {'name':'s', 'id':19},
          {'name':'t', 'id':20},
          {'name':'u', 'id':21},
          {'name':'v', 'id':22},
          {'name':'w', 'id':23},
          {'name':'x', 'id':24},
          {'name':'y', 'id':25},
          {'name':'z', 'id':26},
          ]

In [3]:
with open('Tensorflow\workplace\Annotation\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Generate TF records

In [4]:
!pip install tensorflow-object-detection-api

Defaulting to user installation because normal site-packages is not writeable


In [8]:
import os

if os.name == 'posix':
    !python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
    !python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}
else:
    !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
    !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Traceback (most recent call last):
  File "Tensorflow/scripts/generate_tfrecord.py", line 60, in <module>
    label_map_dict = label_map_util.get_label_map_dict(label_map)
  File "/home/ishani/.local/lib/python3.8/site-packages/object_detection/utils/label_map_util.py", line 164, in get_label_map_dict
    label_map = load_labelmap(label_map_path)
  File "/home/ishani/.local/lib/python3.8/site-packages/object_detection/utils/label_map_util.py", line 133, in load_labelmap
    label_map_string = fid.read()
  File "/home/ishani/.local/lib/python3.8/site-packages/tensorflow/python/lib/io/file_io.py", line 117, in read
    self._preread_check()
  File "/home/ishani/.local/lib/python3.8/site-packages/tensorflow/python/lib/io/file_io.py", line 79, in _preread_check
    self._read_buf = _pywrap_file_io.BufferedInputStream(
TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. tensorflow.python._pywrap_file_io.BufferedInputStream(filename: 